# we are continuing from sensitivity of functions and analysing them


In [0]:
import torch

In [0]:
def get_parallel_db(db, index):
  return torch.cat((db[:index], db[index + 1 :]))

In [0]:
def get_parallel_dbs(db):
  parallel_dbs = list()
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
  return parallel_dbs  

In [0]:
def create_db_and_paralleldbs(entries):
  db = torch.rand(entries) > 0.5
  pdbs = get_parallel_dbs(db)
  return db, pdbs

In [0]:
def query(db, threshold = 5):
  return (db.sum() > threshold).float()

In [0]:
db, pdbs = create_db_and_paralleldbs(10)
db
db.sum()
query(db)

tensor(1.)

In [0]:
def L1_senstivity(query, n_entries):
  db, pdbs = create_db_and_paralleldbs(n_entries)
  maximum_distance = 0
  total_result = query(db)
  for pdb in pdbs:
    current_result = query(pdb)
    current_distance = torch.abs(current_result - total_result)
    if(current_distance > maximum_distance):
      maximum_distance = current_distance
  return maximum_distance 

In [0]:
L1_senstivity(query)

0

In [0]:
for i in range(10):
  print(L1_senstivity(query, 10))

0
0
0
0
0
0
0
tensor(1.)
0
0


## Let's now attack differentially on the database which is called differential attack by various queries


In [0]:
db, pdbs = create_db_and_paralleldbs(20)

In [0]:
db

tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       dtype=torch.uint8)

In [0]:
pdbs[10]

tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       dtype=torch.uint8)

We are going to find the exact value of row 10 in the database by doing two simple queries.
One will be sum over the entire database.
another will be sum over the parallel database not contaning that value of row 10 (here maybe 10 th pbds)
taking subtraction between them will fetch the exact value of the row 10 from the database.

In [0]:
def query(db):
  return db.sum()

In [0]:
first_query = query(db)
second_query = query(pdbs[10])
print(first_query)
print(second_query)

tensor(7)
tensor(7)


In [0]:
first_query - second_query

tensor(0)

In [0]:
#differential attack using addition

sum(db) - sum(pdbs[10])

tensor(0, dtype=torch.uint8)

In [0]:
#differential attack using addition

(sum(db).float() / len(db)) - (sum(pdbs[10]).float() / len(db))

tensor(0.)

In [0]:
#differential attack using threshold
#since we know that threhold here can be sum that is 7 for original database so we apply condition as in greater than 6 will hold true boht and result in the false (0)
(sum(db).float() > 6) - (sum(pdbs[10]).float() > 6)

tensor(0, dtype=torch.uint8)

EXPLORING MORE 

In [0]:
def query(db):
  return db.median()

In [0]:
db, pdbs = create_db_and_paralleldbs(15)

In [21]:
db

tensor([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1], dtype=torch.uint8)

In [22]:
query(db)

tensor(0, dtype=torch.uint8)

In [26]:
for i in range(10):
  print(L1_senstivity(query, 10))

0
0
tensor(1, dtype=torch.uint8)
0
0
tensor(1, dtype=torch.uint8)
0
0
0
tensor(1, dtype=torch.uint8)
